In [1]:
import face_rec
import pandas as pd
import numpy as np
import cv2

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tuand/.insightface\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tuand/.insightface\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


d:\Code\Attendance_System\attendance_system\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [2]:
face_rec.r.ping()

True

### extract data from database

In [3]:
name = 'academy:register'
retrive_dict = face_rec.r.hgetall(name)
retrive_series = pd.Series(retrive_dict)
retrive_series = retrive_series. apply(lambda x: np. frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))
retrive_series.index = index
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role', 'facial_features']
retrive_df[['Name', 'Role']] = retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

,name_role,facial_features,Name,Role
0,Morgan Freeman@Teacher,"[0.4853508, -0.61925495, -0.4069931, 1.0687788...",Morgan Freeman,Teacher
1,Barack Obama@Teacher,"[0.9485806, -0.5584053, 0.094820514, -1.279659...",Barack Obama,Teacher
2,abc@Student,"[-1.1158142, -0.90308434, 0.33440918, -0.31692...",abc,Student
3,Angelina Jolie@Student,"[0.4324205, 1.0748497, -0.128624, -1.8332914, ...",Angelina Jolie,Student
4,Chris Evans@Student,"[0.72017133, -1.468415, -0.7781255, -1.4601074...",Chris Evans,Student
5,Scarlett Johansson@Student,"[0.19131932, -0.4884374, -1.6902335, 0.4423968...",Scarlett Johansson,Student


### real-time prediction

In [4]:
cap = cv2.VideoCapture(0) # default , # 1 for external camera

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    pred_frame = face_rec.face_prediction(frame,retrive_df,'facial_features', ['Name','Role' ], thresh=0.5)
    
    #cv2.imshow('frame', frame)
    cv2.imshow('prediciton',pred_frame)
    
    if cv2.waitKey(1) == 27: # if i press esc button this condition will get trigged
        break

cv2.destroyAllWindows()
cap.release()